In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, count, avg, round, udf
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import DoubleType


In [2]:
spark = SparkSession.builder\
    .appName("Préparation de données")\
    .enableHiveSupport()\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")
spark.catalog.clearCache()
spark.sql("USE concessionnaire")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 20:52:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [3]:
df_catalogue = spark.sql("SELECT * FROM catalogue;")
df_catalogue.show(n=10)

+----------+----------------+-----------+--------+--------+-------------------+-----------+----------+------------+
|    marque|          modele|   longueur|nbplaces|nbportes|          categorie|bonus_malus|rejets_co2|cout_energie|
+----------+----------------+-----------+--------+--------+-------------------+-----------+----------+------------+
|   renault|vel satis 3.5 v6|tres longue|       5|       5|      SUV/Crossover|    -6000.0|       0.0|       206.0|
|    lancia| ypsilon 1.4 16v|     courte|       5|       3|citadine economique|    -4865.0|      33.0|       158.0|
|volkswagen|     polo 1.2 6v|     courte|       5|       3|citadine economique|    -6000.0|      23.0|        96.0|
|  daihatsu|       cuore 1.0|     courte|       5|       3|citadine economique|    -4865.0|      33.0|       158.0|
|     smart|      toledo 1.6|     longue|       5|       5|          familiale|    -6000.0|       0.0|       191.0|
|   renault|vel satis 3.5 v6|tres longue|       5|       5|             

### Verification des doublons

In [4]:
doublons = df_catalogue.groupBy(*df_catalogue.columns).agg(count("*").alias("count")).filter("count > 1")
doublons.show(truncate=False)

+------+------+--------+--------+--------+---------+-----------+----------+------------+-----+
|marque|modele|longueur|nbplaces|nbportes|categorie|bonus_malus|rejets_co2|cout_energie|count|
+------+------+--------+--------+--------+---------+-----------+----------+------------+-----+
+------+------+--------+--------+--------+---------+-----------+----------+------------+-----+



### Analyse des **null**

In [5]:
df_catalogue.select(
    *[sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_catalogue.columns]
).show()

+------+------+--------+--------+--------+---------+-----------+----------+------------+
|marque|modele|longueur|nbplaces|nbportes|categorie|bonus_malus|rejets_co2|cout_energie|
+------+------+--------+--------+--------+---------+-----------+----------+------------+
|     0|     0|       0|       0|       0|        0|          0|         0|           0|
+------+------+--------+--------+--------+---------+-----------+----------+------------+



### Normalisation de la **marque**

In [6]:
indexer_marque = StringIndexer(inputCol="marque", outputCol="marque_index")
indexer_model_marque = indexer_marque.fit(df_catalogue)
df_catalogue = indexer_model_marque.transform(df_catalogue)

encoder = OneHotEncoder(inputCol="marque_index", outputCol="marque_encoded")
df_catalogue = encoder.fit(df_catalogue).transform(df_catalogue)

df_catalogue = df_catalogue.drop('marque') 
df_catalogue = df_catalogue.drop('marque_index') 

df_catalogue = df_catalogue.withColumnRenamed('marque_encoded', 'marque')

df_catalogue.show(n=10)

+----------------+-----------+--------+--------+-------------------+-----------+----------+------------+---------------+
|          modele|   longueur|nbplaces|nbportes|          categorie|bonus_malus|rejets_co2|cout_energie|         marque|
+----------------+-----------+--------+--------+-------------------+-----------+----------+------------+---------------+
|vel satis 3.5 v6|tres longue|       5|       5|      SUV/Crossover|    -6000.0|       0.0|       206.0| (19,[0],[1.0])|
| ypsilon 1.4 16v|     courte|       5|       3|citadine economique|    -4865.0|      33.0|       158.0|(19,[14],[1.0])|
|     polo 1.2 6v|     courte|       5|       3|citadine economique|    -6000.0|      23.0|        96.0| (19,[1],[1.0])|
|       cuore 1.0|     courte|       5|       3|citadine economique|    -4865.0|      33.0|       158.0|(19,[10],[1.0])|
|      toledo 1.6|     longue|       5|       5|          familiale|    -6000.0|       0.0|       191.0|(19,[18],[1.0])|
|vel satis 3.5 v6|tres longue|  

### Analyse de **Modele**

In [7]:
count_modele = df_catalogue.select("modele").distinct().count()
print(f"Il y a {count_modele} modeles.")

Il y a 32 modeles.


On travail uniquement sur les catégories, les modèles spécifiques n'ajoutent pas de valeur.
L'analyse est simplifier en se concentrant sur des regroupements plus larges.

In [8]:
indexer = StringIndexer(inputCol="categorie", outputCol="categorie_indexed")
indexer_model = indexer.fit(df_catalogue)
df_catalogue = indexer_model.transform(df_catalogue)

df_catalogue = df_catalogue.drop('modele') 
df_catalogue = df_catalogue.drop('categorie') 

df_catalogue = df_catalogue.withColumnRenamed('categorie_indexed', 'categorie')

df_catalogue.show(n=10)

+-----------+--------+--------+-----------+----------+------------+---------------+---------+
|   longueur|nbplaces|nbportes|bonus_malus|rejets_co2|cout_energie|         marque|categorie|
+-----------+--------+--------+-----------+----------+------------+---------------+---------+
|tres longue|       5|       5|    -6000.0|       0.0|       206.0| (19,[0],[1.0])|      1.0|
|     courte|       5|       3|    -4865.0|      33.0|       158.0|(19,[14],[1.0])|      2.0|
|     courte|       5|       3|    -6000.0|      23.0|        96.0| (19,[1],[1.0])|      2.0|
|     courte|       5|       3|    -4865.0|      33.0|       158.0|(19,[10],[1.0])|      2.0|
|     longue|       5|       5|    -6000.0|       0.0|       191.0|(19,[18],[1.0])|      0.0|
|tres longue|       5|       5|    -6000.0|       0.0|       206.0| (19,[0],[1.0])|      3.0|
|     courte|       5|       5|    -6000.0|      22.0|       126.0|(19,[15],[1.0])|      3.0|
|     courte|       5|       5|    -6000.0|      13.0|      

### Analyse de **longueur**

In [9]:
df_catalogue.select("longueur").distinct().show()

+-----------+
|   longueur|
+-----------+
|tres longue|
|     courte|
|     longue|
|    moyenne|
+-----------+



In [10]:
indexer_longueur = StringIndexer(inputCol="longueur", outputCol="longueur_indexed")
indexer_model_longueur = indexer_longueur.fit(df_catalogue)
df_catalogue = indexer_model_longueur.transform(df_catalogue)

df_catalogue = df_catalogue.drop('longueur')
df_catalogue = df_catalogue.withColumnRenamed('longueur_indexed', 'longueur')

df_catalogue.show(n=4)

+--------+--------+-----------+----------+------------+---------------+---------+--------+
|nbplaces|nbportes|bonus_malus|rejets_co2|cout_energie|         marque|categorie|longueur|
+--------+--------+-----------+----------+------------+---------------+---------+--------+
|       5|       5|    -6000.0|       0.0|       206.0| (19,[0],[1.0])|      1.0|     3.0|
|       5|       3|    -4865.0|      33.0|       158.0|(19,[14],[1.0])|      2.0|     2.0|
|       5|       3|    -6000.0|      23.0|        96.0| (19,[1],[1.0])|      2.0|     2.0|
|       5|       3|    -4865.0|      33.0|       158.0|(19,[10],[1.0])|      2.0|     2.0|
+--------+--------+-----------+----------+------------+---------------+---------+--------+
only showing top 4 rows



## Classificateur

In [11]:
assembler = VectorAssembler(
    inputCols=["nbplaces", "nbportes", "bonus_malus", "rejets_co2", "cout_energie", "marque", "longueur"],
    outputCol="features")

df_final = assembler.transform(df_catalogue)

(trainingData, testData) = df_final.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestClassifier(labelCol="categorie", featuresCol="features", numTrees=20)
model = rf.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(
    labelCol="categorie", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)
print("Précision du modèle sur l'ensemble de test = {:.2f}%".format(accuracy * 100))

# Conversion des prédictions en labels de catégories
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedCategorie", labels=indexer_model.labels)
predictions = labelConverter.transform(predictions)

# Affichage des prédictions avec les catégories réelles et prédites
predictions.select("categorie", "predictedCategorie", "features").show()

Précision du modèle sur l'ensemble de test = 50.00%
+---------+-------------------+--------------------+
|categorie| predictedCategorie|            features|
+---------+-------------------+--------------------+
|      1.0|          familiale|(25,[0,1,2,4,5,24...|
|      2.0|citadine economique|(25,[0,1,2,3,4,15...|
|      0.0|          familiale|(25,[0,1,2,3,4,6,...|
|      3.0|          familiale|(25,[0,1,2,3,4,8,...|
|      0.0|          familiale|(25,[0,1,2,3,4,18...|
|      3.0|          familiale|(25,[0,1,2,3,4,7,...|
+---------+-------------------+--------------------+



In [ ]:
dt = DecisionTreeClassifier(labelCol="categorie", featuresCol="features")
model_dt = dt.fit(trainingData)
predictions_dt = model_dt.transform(testData)

accuracy_dt = evaluator.evaluate(predictions_dt)
print("Précision du modèle Decision Tree sur l'ensemble de test = {:.2f}%".format(accuracy_dt * 100))

Précision du modèle Decision Tree sur l'ensemble de test = 50.00%


In [14]:
df_catalogue.groupBy('categorie').count().show()

# Taille du jeu de données total
print(f"Taille du jeu de données : {df_final.count()}")

# Taille des ensembles d'entraînement et de test
print(f"Taille de l'ensemble d'entraînement : {trainingData.count()}")
print(f"Taille de l'ensemble de test : {testData.count()}")

+---------+-----+
|categorie|count|
+---------+-----+
|      1.0|    6|
|      2.0|    6|
|      0.0|   19|
|      3.0|    5|
+---------+-----+

Taille du jeu de données : 36
Taille de l'ensemble d'entraînement : 30
Taille de l'ensemble de test : 6
